In [1]:
from ts_modelling.simple_experiment import SimpleExp
from argparse import Namespace

In [2]:
import yaml
with open('args.yaml', 'r') as file:
    config = yaml.safe_load(file)
args = Namespace(**config)
exp = SimpleExp(args)


this is a changer version
Use CPU


AttributeError: 'torch.device' object has no attribute '_apply'

9